# Setup

In [1]:
import time
from bs4 import BeautifulSoup
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd

/var/folders/86/htttx7197fv90v3w5v6m_s_c0000gn/T/ipykernel_45092/4065996874.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
load_dotenv()
client = OpenAI()

# Read product URLs

In [ ]:
# Read the specbook.csv from 01_llmpipeline directory
df = pd.read_csv('01_llmpipeline/specbook.csv', index_col='category')
df['id'] = range(1, len(df) + 1)
print(f"Loaded {len(df)} rows from specbook.csv")
df.head()

# for url, id in zip(df['product_url'], df['id']):
#     print(url, id)

Loaded 87 rows from specbook.csv
https://www.dunnedwards.com/colors/browser/dew340 1
https://www.coloratelierpaint.com/product-page/tadelakt-shower-plaster 2
https://www.dunnedwards.com/colors/browser/dew340 3
https://www.dunnedwards.com/colors/browser/dew340 4
https://www.strongarhardware.com/extra-length-ladder-style-back-to-back-push-pull-door-handle-matte-black-powder-coated-stainless-steel-finish/ 5
https://www.kwikset.com/products/detail/milan-lever-round-bed-bath?variant=155mil-rdt-514 6
https://www.kwikset.com/products/detail/milan-lever-round-hall-closet?variant=154mil-rdt-514 7
https://emtek.com/door-hinges/4x4squarebarrelheavydutyhinges 8
https://www.msisurfaces.com/marble/calacatta-marble/#:~:text=Calacatta%20Gold%20marble%20is%20a,islands%2C%20marble%20floors%20and%20backsplashes. 9
https://www.cosentino.com/colors/silestone/charcoal-soapstone/ 10
https://www.caesarstoneus.com/catalog/1141-pure-white/ 11
https://www.fireclaytile.com/tile/colors/detail/daisy/tile-field-2-x-

In [4]:
unique = unique=df['product_url'].nunique()
print(f"Unique product urls: {unique}")

# Count the number of rows where product_url is not null
not_null = df['product_url'].notna().sum()
print(f"Number of rows where product_url is not null: {not_null}")

# Count the number of rows where product_url is null
null = df['product_url'].isna().sum()
print(f"Number of rows where product_url is null: {null}")

product_urls = df["product_url"].to_list()

Unique product urls: 82
Number of rows where product_url is not null: 87
Number of rows where product_url is null: 0


# Fetch Product HTML

In [5]:
from concurrent.futures import ThreadPoolExecutor
from tools.stealth_scraper import StealthScraper

scraper = StealthScraper()

results = {}

with ThreadPoolExecutor(max_workers=10) as executor:
    for url, result in zip(product_urls, executor.map(scraper.scrape_url, product_urls)):
        results[url] = result

In [6]:
# Create lists to store the data
data = []

for url, result in results.items():
    # Extract the basic fields
    row = {
        'url': url,
        'success': result.success,
        'content_length': len(result.content) if result.content else 0,
        'status_code': result.status_code,
        'final_method': result.final_method,
        'error_reason': result.error_reason,
        'page_issues': result.page_issues,
        # Convert the full result to JSON for storage
        'full_result': result.json()
    }
    data.append(row)

# Create the DataFrame
result_df = pd.DataFrame(data)

# Reorder columns to match requested order
column_order = ['success', 'content_length', 'status_code', 'final_method', 'url', 'full_result',  'error_reason', 'page_issues']
result_df = result_df[column_order]

/var/folders/86/htttx7197fv90v3w5v6m_s_c0000gn/T/ipykernel_45092/2814656682.py:15: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  'full_result': result.json()


In [7]:
print(result_df["status_code"].value_counts())
print(result_df["page_issues"].value_counts())
print(result_df["success"].value_counts())

status_code
200    65
500    13
404     4
Name: count, dtype: int64
page_issues
[]                          70
[PageIssue.BOT_DETECTED]    12
Name: count, dtype: int64
success
True     65
False    17
Name: count, dtype: int64


In [132]:
soups = {}

with open(f'01_llmpipeline/1-requests_failed_{int(time.time())}.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['url', 'status_code', 'text'])

    for url, response in results.items():
        soup = BeautifulSoup(response.get("text"), "html.parser")

        code = response.get("code")
        if code != 200:
            writer.writerow([url, code, soup.text])
        else:
            soups[url] = soup

# Preprocess HTML

In [140]:
REMOVE_TAGS = [
    "script", "style", "noscript", "svg", "footer", "header",
    "nav", "form", "iframe", "aside", "canvas", "button", "input", "select", "option"
]

GARBAGE_KEYWORDS = ["cookie", "newsletter", "subscribe", "banner", "social", "share", "advert"]

preprocessed_html = {}

for url, soup in soups.items():

    # Remove noise tags
    for tag in soup(REMOVE_TAGS):
        tag.decompose()

    # # Remove elements with garbage classes/ids
    # for el in soup.find_all(attrs={"class": True}):
    #     cls = " ".join(el.get("class"))
    #     if any(kw in cls.lower() for kw in GARBAGE_KEYWORDS):
    #         el.decompose()
    #
    # for el in soup.find_all(attrs={"id": True}):
    #     id_ = el.get("id")
    #     if id_ and any(kw in id_.lower() for kw in GARBAGE_KEYWORDS):
    #         el.decompose()

    # Extract visible text
    text = soup.get_text(separator="\n", strip=True)
    text_lines = [line.strip() for line in text.splitlines() if line.strip()]
    visible_text = "\n".join(text_lines)

    # Extract metadata
    metadata = {
        tag.get("property") or tag.get("name"): tag.get("content")
        for tag in soup.find_all("meta")
        if tag.get("content")
    }

    # Extract images with alt text
    images = []
    for img in soup.find_all("img"):
        src = img.get("src")
        alt = img.get("alt", "").strip()
        if src:
            images.append({"src": src, "alt": alt})

    preprocessed_html[url] = {
        "title": soup.title.string.strip() if soup.title and soup.title.string else "",
        "metadata": metadata,
        "text": visible_text,
        "images": images
    }


In [ ]:
import json
# Max characters after pre-process
print(max([len(json.dumps(x)) for x in preprocessed_html]))

with open('01_llmpipeline/2-preprocessed_html.json', 'w') as json_file:
    json.dump(preprocessed_html, json_file)

# with open('preprocessed_html.json', 'r') as json_file:
#     preprocessed_html = json.load(json_file)

# Create Prompts

In [207]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """
You are a project architect tasked with fetching specification details from the following product website's HTML page. Extract the relevant product information for documentation in a specification book.

If you are not 99.9% sure that the information is correct, return the value with the highest probability, including the probability in the value field.

TITLE:
data['title']

PRODUCT URL:
{product_url}

METADATA:
data['metadata']

TEXT CONTENT:
data['text']

IMAGES:
data['images']

Extract the following structured data in JSON format from the provided product web page:

- image_url: Direct URL to the product image. If the url is a relative url, append to the base_url **without modifying the url as found in the html <img> tag**. Prioritize <img> tag urls that have an `id` attribute or any attribute with a value that relates it to the main product image.
- type: The product category (e.g. range hood, grill, fireplace, etc.).
- description: Short product description, including brand, size, material, color, and any notable features
- model_no: Manufacturer model number, item no, or sku no.
- product_link: Original product page URL.
- qty: Quantity if specified; otherwise return "unspecified".
- key: A unique reference key (if available).

Return your output in this JSON format without the json tag and codeblocks.
You **don’t add extra formatting instructions yourself**:

```json
{{
  "image_url": "",
  "type": "",
  "description": "",
  "model_no": "",
  "product_link": "",
  "qty": "",
  "key": ""
}}
```

data:
{data}
"""

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template)]
)

In [208]:
prompts = {}

for url, website_data in preprocessed_html.items():
    prompts[url] = prompt_template.invoke({"data": json.dumps(website_data), "product_url": url})

In [209]:
with open(f'01_llmpipeline/3-prompts-{int(time.time())}.csv', mode='w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['url', 'prompt'])

    for url, prompt in prompts.items():
        writer.writerow([url, prompt.to_string()])


# Invoke Model

In [210]:
gpt_4o_mini = init_chat_model("gpt-4o-mini", model_provider="openai")

In [202]:
# product_url = "https://www.subzero-wolf.com/sub-zero/full-size-refrigeration/integrated-fridges/36-inch-integrated-column-freezer-ice-maker-panel-ready"
# # product_url = "https://www.subzero-wolf.com/sub-zero/full-size-refrigeration/integrated-fridges/36-inch-integrated-column-refrigerator-internal-dispenser-panel-ready"
# oneshot = preprocessed_html[product_url]
# oneshot_p = prompt_template.invoke({"data": json.dumps(oneshot), "product_url": product_url})
# c = gpt_4o_mini.invoke(oneshot_p)
#

In [203]:
# print(json.loads(c.content)['image_url'])

In [ ]:
llm_responses = {}

for url, prompt in prompts.items():
    llm_responses[url] = gpt_4o_mini.invoke(prompt)

In [212]:
with open(f'01_llmpipeline/4-llm-{int(time.time())}.csv', mode='w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['url', 'llm_message'])

    for url, llm_message in llm_responses.items():
        writer.writerow([url, llm_message.content])

In [ ]:
for url, llm_message in llm_responses.items():
    print(url, json.loads(llm_message.content)["image_url"], "\n")

aa# Evaluation

In [ ]:
evaluator = ProductExtractionEvaluator()
batch_results = evaluator.evaluate_batch(evaluations)

# Generate Specbook

In [81]:
fieldnames = ['image_url', 'type', 'description', 'model_no']

with open('01_llmpipeline/5-specbook.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader() # Writes the header row
    writer.writerows(filtered_list)